In [ ]:
import coco_dataset
import pathlib
import show_utils
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
import model
import common

In [ ]:
inp_size = (368, 368)
targ_size = (46, 46)
coco_dataset = coco_dataset.CocoKeypoints(
    root=str(pathlib.Path("../coco") / "images" / "train2017"),
    annFile=str(pathlib.Path("../coco") / "annotations" / "annotations" / "person_keypoints_train2017.json"),
    input_transform=transforms.Compose([
        transforms.Resize(inp_size),
        transforms.ToTensor(),
        transforms.ConvertImageDtype(torch.float32),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]),
    targ_size=targ_size)
i = 0

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    device = "cuda:0"
    model = torch.nn.DataParallel(model.openpose(), device_ids=[0])
    model.load_state_dict(torch.load("save_modelGG.pth"))
    model.eval()
print(device)

In [ ]:
i = 5

In [ ]:
sample = coco_dataset[i]

input_image, pafs, heatmaps, mask_out, keypoints, image, target = sample
show_utils.show_coco(image, target, coco_dataset.coco, draw_bbox=False)

input_image.unsqueeze_(0)
pred_pafs, pred_heatmaps, _, _ = model(input_image.to(device))  

pred_pafs.squeeze_(0)
pred_heatmaps.squeeze_(0)
print(f"sample: {i}")
i += 1

In [ ]:
import post
image_size = image.size

with torch.no_grad():
    kpt_groups = post.post_process(pred_heatmaps.cpu(), pred_pafs.cpu())
if kpt_groups:
    keypoints_post = post.coco_format(kpt_groups, image_size)
    show_utils.draw_keypoints(image,keypoints_post, connectivity=common.connect_skeleton, keypoint_color="orange", line_color="magenta")
else:
    print("NO DETECTIONS")

In [ ]:
# %matplotlib widget
# with torch.no_grad():

    # show_utils.show_heatmaps_combined(pred_heatmaps.cpu())

    # show_utils.show_heatmaps(pred_heatmaps.cpu())

    # show_utils.show_pafs(pred_pafs.cpu())

    # show_utils.show_pafs_quiver(pred_pafs.cpu(), keypoints, size=targ_size)

    # plt.figure()
    # show_utils.show_pafs_quiver_combined(pred_pafs.cpu(), size=targ_size)

    # show_utils.show_tensors(mask_out)